In [15]:
import pandas as pd
import numpy as np
import os.path
import math
import collections
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from keras.utils import np_utils
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.ensemble import GradientBoostingRegressor
import math
import csv
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV
import urllib
from sklearn.svm import LinearSVC

In [16]:
!pip install binarytree

In [17]:
!pip install bracketeer

# Load all the Data.


In [18]:
reg_season_compact = pd.read_csv('./DataFiles//RegularSeasonCompactResults.csv')
seasons = pd.read_csv('./DataFiles/Seasons.csv')
teams = pd.read_csv('./DataFiles/Teams.csv')
tourney_compact = pd.read_csv('./DataFiles/NCAATourneyCompactResults.csv')
submission = pd.read_csv('./SampleSubmissionStage1.csv')
tourney_seeds = pd.read_csv('./DataFiles/NCAATourneySeeds.csv')
team_conferences = pd.read_csv('./DataFiles/TeamConferences.csv')



In [19]:
teamList = teams['TeamName'].tolist()

# Clean and Manipulate the Data.


In [20]:
def check_Power6(team_id):
    team = team_conferences[(team_conferences['Season'] == 2018) & (team_conferences['TeamID'] == team_id)]
    # Can't find the team
    if (len(team) == 0):
        return 0
    conf_name = team.iloc[0]['ConfAbbrev']
    return int(conf_name == 'SEC' or conf_name == 'ACC'or conf_name == 'BIG_10' or conf_name == 'BIG_TEWELVE' or conf_name == 'BIG_EAST' or conf_name == 'PAC_12')

def get_team_ID(name):
    return teams[teams['TeamName'] == name].values[0][0]

def get_team_name(team_id):
    return teams[teams['TeamID'] == team_id].values[0][1]

def list_for_URL(team_list):
    team_list = [x.lower() for x in team_list]
    team_list = [t.replace(' ', '-') for t in team_list]
    team_list = [t.replace('st', 'state') for t in team_list]
    team_list = [t.replace('northern-dakota', 'north-dakota') for t in team_list]
    team_list = [t.replace('nc-', 'north-carolina-') for t in team_list]
    team_list = [t.replace('fl-', 'florida-') for t in team_list]
    team_list = [t.replace('ga-', 'georgia-') for t in team_list]
    team_list = [t.replace('lsu', 'louisiana-state') for t in team_list]
    team_list = [t.replace('maristate', 'marist') for t in team_list]
    team_list = [t.replace('stateate', 'state') for t in team_list]
    team_list = [t.replace('northernorthern', 'northern') for t in team_list]
    team_list = [t.replace('usc', 'southern-california') for t in team_list]
    base = 'http://www.sports-reference.com/cbb/schools/'
    for team in team_list:
        url = base + team + '/'
list_for_URL(teamList);

def handle_cases(arr):
    indices = []
    list_len = len(arr)
    for i in range(list_len):
        if (arr[i] == 'St' or arr[i] == 'FL'):
            indices.append(i)
    for p in indices:
        arr[p-1] = arr[p-1] + ' ' + arr[p]
    for i in range(len(indices)):
        arr.remove(arr[indices[i] - i])
    return arr


def get_tourney_appearances(team_id):
    return len(tourney_seeds[tourney_seeds['TeamID'] == team_id].index)

def different_csv(df):
    # The stats CSV is a lit different in terms of naming so below is just some data cleaning
    df['School'] = df['School'].replace('(State)', 'St', regex=True)
    df['School'] = df['School'].replace('Albany (NY)', 'Albany NY')
    df['School'] = df['School'].replace('Boston University', 'Boston Univ')
    df['School'] = df['School'].replace('Central Michigan', 'C Michigan')
    df['School'] = df['School'].replace('(Eastern)', 'E', regex=True)
    df['School'] = df['School'].replace('Louisiana St', 'LSU')
    df['School'] = df['School'].replace('North Carolina St', 'NC State')
    df['School'] = df['School'].replace('Southern California', 'USC')
    df['School'] = df['School'].replace('University of California', 'California', regex=True)
    df['School'] = df['School'].replace('American', 'American Univ')
    df['School'] = df['School'].replace('Arkansas-Little Rock', 'Ark Little Rock')
    df['School'] = df['School'].replace('Arkansas-Pine Bluff', 'Ark Pine Bluff')
    df['School'] = df['School'].replace('Bowling Green St', 'Bowling Green')
    df['School'] = df['School'].replace('Brigham Young', 'BYU')
    df['School'] = df['School'].replace('Cal Poly', 'Cal Poly SLO')
    df['School'] = df['School'].replace('Centenary (LA)', 'Centenary')
    df['School'] = df['School'].replace('Central Connecticut St', 'Central Conn')
    df['School'] = df['School'].replace('Charleston Southern', 'Charleston So')
    df['School'] = df['School'].replace('Coastal Carolina', 'Coastal Car')
    df['School'] = df['School'].replace('College of Charleston', 'Col Charleston')
    df['School'] = df['School'].replace('Cal St Fullerton', 'CS Fullerton')
    df['School'] = df['School'].replace('Cal St Sacramento', 'CS Sacramento')
    df['School'] = df['School'].replace('Cal St Bakersfield', 'CS Bakersfield')
    df['School'] = df['School'].replace('Cal St Northridge', 'CS Northridge')
    df['School'] = df['School'].replace('East Tennessee St', 'ETSU')
    df['School'] = df['School'].replace('Detroit Mercy', 'Detroit')
    df['School'] = df['School'].replace('Fairleigh Dickinson', 'F Dickinson')
    df['School'] = df['School'].replace('Florida Atlantic', 'FL Atlantic')
    df['School'] = df['School'].replace('Florida Gulf Coast', 'FL Gulf Coast')
    df['School'] = df['School'].replace('Florida International', 'Florida Intl')
    df['School'] = df['School'].replace('George Washington', 'G Washington')
    df['School'] = df['School'].replace('Georgia Southern', 'Ga Southern')
    df['School'] = df['School'].replace('Gardner-Webb', 'Gardner Webb')
    df['School'] = df['School'].replace('Illinois-Chicago', 'IL Chicago')
    df['School'] = df['School'].replace('Kent St', 'Kent')
    df['School'] = df['School'].replace('Long Island University', 'Long Island')
    df['School'] = df['School'].replace('Loyola Marymount', 'Loy Marymount')
    df['School'] = df['School'].replace('Loyola (MD)', 'Loyola MD')
    df['School'] = df['School'].replace('Loyola (IL)', 'Loyola-Chicago')
    df['School'] = df['School'].replace('Massachusetts', 'MA Lowell')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    df['School'] = df['School'].replace('Miami (FL)', 'Miami FL')
    df['School'] = df['School'].replace('Miami (OH)', 'Miami OH')
    df['School'] = df['School'].replace('Missouri-Kansas City', 'Missouri KC')
    df['School'] = df['School'].replace('Monmouth', 'Monmouth NJ')
    df['School'] = df['School'].replace('Mississippi Valley St', 'MS Valley St')
    df['School'] = df['School'].replace('Montana St', 'MTSU')
    df['School'] = df['School'].replace('Northern Colorado', 'N Colorado')
    df['School'] = df['School'].replace('North Dakota St', 'N Dakota St')
    df['School'] = df['School'].replace('Northern Illinois', 'N Illinois')
    df['School'] = df['School'].replace('Northern Kentucky', 'N Kentucky')
    df['School'] = df['School'].replace('North Carolina A&T', 'NC A&T')
    df['School'] = df['School'].replace('North Carolina Central', 'NC Central')
    df['School'] = df['School'].replace('Pennsylvania', 'Penn')
    df['School'] = df['School'].replace('South Carolina St', 'S Carolina St')
    df['School'] = df['School'].replace('Southern Illinois', 'S Illinois')
    df['School'] = df['School'].replace('UC-Santa Barbara', 'Santa Barbara')
    df['School'] = df['School'].replace('Southeastern Louisiana', 'SE Louisiana')
    df['School'] = df['School'].replace('Southeast Missouri St', 'SE Missouri St')
    df['School'] = df['School'].replace('Stephen F. Austin', 'SF Austin')
    df['School'] = df['School'].replace('Southern Methodist', 'SMU')
    df['School'] = df['School'].replace('Southern Mississippi', 'Southern Miss')
    df['School'] = df['School'].replace('Southern', 'Southern Univ')
    df['School'] = df['School'].replace('St. Bonaventure', 'St Bonaventure')
    df['School'] = df['School'].replace('St. Francis (NY)', 'St Francis NY')
    df['School'] = df['School'].replace('Saint Francis (PA)', 'St Francis PA')
    df['School'] = df['School'].replace('St. John\'s (NY)', 'St John\'s')
    df['School'] = df['School'].replace('Saint Joseph\'s', 'St Joseph\'s PA')
    df['School'] = df['School'].replace('Saint Louis', 'St Louis')
    df['School'] = df['School'].replace('Saint Mary\'s (CA)', 'St Mary\'s CA')
    df['School'] = df['School'].replace('Mount Saint Mary\'s', 'Mt St Mary\'s')
    df['School'] = df['School'].replace('Saint Peter\'s', 'St Peter\'s')
    df['School'] = df['School'].replace('Texas A&M-Corpus Christian', 'TAM C. Christian')
    df['School'] = df['School'].replace('Texas Christian', 'TCU')
    df['School'] = df['School'].replace('Tennessee-Martin', 'TN Martin')
    df['School'] = df['School'].replace('Texas-Rio Grande Valley', 'UTRGV')
    df['School'] = df['School'].replace('Texas Southern', 'TX Southern')
    df['School'] = df['School'].replace('Alabama-Birmingham', 'UAB')
    df['School'] = df['School'].replace('UC-Davis', 'UC Davis')
    df['School'] = df['School'].replace('UC-Irvine', 'UC Irvine')
    df['School'] = df['School'].replace('UC-Riverside', 'UC Riverside')
    df['School'] = df['School'].replace('Central Florida', 'UCF')
    df['School'] = df['School'].replace('Louisiana-Lafayette', 'ULL')
    df['School'] = df['School'].replace('Louisiana-Monroe', 'ULM')
    df['School'] = df['School'].replace('Maryland-Baltimore County', 'UMBC')
    df['School'] = df['School'].replace('North Carolina-Asheville', 'UNC Asheville')
    df['School'] = df['School'].replace('North Carolina-Greensboro', 'UNC Greensboro')
    df['School'] = df['School'].replace('North Carolina-Wilmington', 'UNC Wilmington')
    df['School'] = df['School'].replace('Nevada-Las Vegas', 'UNLV')
    df['School'] = df['School'].replace('Texas-Arlington', 'UT Arlington')
    df['School'] = df['School'].replace('Texas-San Antonio', 'UT San Antonio')
    df['School'] = df['School'].replace('Texas-El Paso', 'UTEP')
    df['School'] = df['School'].replace('Virginia Commonwealth', 'VA Commonwealth')
    df['School'] = df['School'].replace('Western Carolina', 'W Carolina')
    df['School'] = df['School'].replace('Western Illinois', 'W Illinois')
    df['School'] = df['School'].replace('Western Kentucky', 'WKU')
    df['School'] = df['School'].replace('Western Michigan', 'W Michigan')
    df['School'] = df['School'].replace('Abilene Christian', 'Abilene Chr')
    df['School'] = df['School'].replace('Montana State', 'Montana St')
    df['School'] = df['School'].replace('Central Arkansas', 'Cent Arkansas')
    df['School'] = df['School'].replace('Houston Baptist', 'Houston Bap')
    df['School'] = df['School'].replace('South Dakota St', 'S Dakota St')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    return df

def home_advantage(row):
    if (row == 'H'):
        home = 1
    if (row == 'A'):
        home = -1
    if (row == 'N'):
        home = 0
    return home

def compare(id_1, id_2, year):
    team_1 = get_season_data(id_1, year)
    team_2 = get_season_data(id_2, year)
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

def input_normalizer(arr):
    for i in range(arr.shape[1]):
        min_val = min(arr[:,i])
        max_val = max(arr[:,i])
        arr[:,i] =  (arr[:,i] - min_val) / (max_val - min_val)
    return arr

def input_normalizer_2(X):
    return (X - np.mean(X, axis = 0)) / np.std(X, axis = 0)

In [21]:
def get_season_data(team_id, year):
    # The data frame below holds stats for every single game in the given year
    year_data = reg_season_compact[reg_season_compact['Season'] == year]
    # Finding number of points per game
    games_won = year_data[year_data.WTeamID == team_id] 
    total_points_scored = games_won['WScore'].sum()
    games_lost = year_data[year_data.LTeamID == team_id] 
    total_games = games_won.append(games_lost)
    num_games = len(total_games.index)
    total_points_scored += games_lost['LScore'].sum()
    
    # Finding number of points per game allowed
    total_points_allowed = games_won['LScore'].sum()
    total_points_allowed += games_lost['WScore'].sum()
    
    #Result of data from http://www.sports-reference.com/cbb
    stats_SOS = pd.read_csv('./MM_Stats/MM_Stats_'+str(year)+'.csv')
    stats_SOS = different_csv(stats_SOS)
    rankings = pd.read_csv('./Rankings/Rankings_'+str(year)+'.csv')
    rankings = different_csv(rankings)
    
    name = get_team_name(team_id)
    team = stats_SOS[stats_SOS['School'] == name]
    team_ranking = rankings[rankings['School'] == name]
    if (len(team.index) == 0 or len(team_ranking.index) == 0): #Can't find the team
        total_3s_made = 0
        total_turnovers = 0
        total_assists = 0
        sos = 0
        total_rebounds = 0
        srs = 0
        total_steals = 0
    else:
        total_3s_made = team['X3P'].values[0]
        total_turnovers = team['TOV'].values[0]
        if (math.isnan(total_turnovers)):
            total_turnovers = 0
        total_assists = team['AST'].values[0]
        if (math.isnan(total_assists)):
            total_assists = 0
        sos = team['SOS'].values[0]
        srs = team['SRS'].values[0]
        total_rebounds = team['TRB'].values[0]
        if (math.isnan(total_rebounds)):
            total_rebounds = 0
        total_steals = team['STL'].values[0]
        if (math.isnan(total_steals)):
            total_steals = 0
    
    #Finding tournament seed for that year
    tourney_year = tourney_seeds[tourney_seeds['Season'] == year]
    seed = tourney_year[tourney_year['TeamID'] == team_id]
    if (len(seed.index) != 0):
        seed = seed.values[0][1]
        tournament_seed = int(seed[1:3])
    else:
        tournament_seed = 25 #Not sure how to represent if a team didn't make the tourney
    
    # Finding number of wins and losses
    num_wins = len(games_won.index)
    # There are some teams who may have dropped to Division 2, so they won't have games 
    # a certain year. In this case, we don't want to divide by 0, so we'll just set the
    # averages to 0 instead
    if num_games == 0:
        avg_points_scored = 0
        avg_points_allowed = 0
        avg_3s_made = 0
        avg_turnovers = 0
        avg_assists = 0
        avg_rebounds = 0
        avg_steals = 0
    else:
        avg_points_scored = total_points_scored/num_games
        avg_points_allowed = total_points_allowed/num_games
        avg_3s_made = total_3s_made/num_games
        avg_turnovers = total_turnovers/num_games
        avg_assists = total_assists/num_games
        avg_rebounds = total_rebounds/num_games
        avg_steals = total_steals/num_games
    return [num_wins, avg_points_scored, avg_points_allowed, check_Power6(team_id), avg_3s_made, avg_assists, avg_turnovers,
           tournament_seed, sos, srs, avg_rebounds, avg_steals, get_tourney_appearances(team_id)]

def create_season_dict(year):
    season_dictionary = collections.defaultdict(list)
    for team in teamList:
        team_id = teams[teams['TeamName'] == team].values[0][0]
        team_vector = get_season_data(team_id, year)
        season_dictionary[team_id] = team_vector
    return season_dictionary

def train_test(years, save_years):
    total_num_games = 0
    for year in years:
        season = reg_season_compact[reg_season_compact['Season'] == year]
        total_num_games += len(season.index)
        tourney = tourney_compact[tourney_compact['Season'] == year]
        total_num_games += len(tourney.index)
    num_features = len(get_season_data(1181,2012)) #Just choosing a random team and seeing the dimensionality of the vector
    xTrain = np.zeros(( total_num_games, num_features + 1))
    yTrain = np.zeros(( total_num_games ))
    index_counter = 0
    for year in years:
        team_vectors = create_season_dict(year)
        season = reg_season_compact[reg_season_compact['Season'] == year]
        num_games_in_season = len(season.index)
        tourney = tourney_compact[tourney_compact['Season'] == year]
        num_games_in_season += len(tourney.index)
        xTrainSeason = np.zeros(( num_games_in_season, num_features + 1))
        yTrainSeason = np.zeros(( num_games_in_season ))
        counter = 0
        for index, row in season.iterrows():
            winning_team = row['WTeamID']
            winning_vector = team_vectors[winning_team]
            losing_team = row['LTeamID']
            losing_vector = team_vectors[losing_team]
            diff = [a - b for a, b in zip(winning_vector, losing_vector)]
            home = home_advantage(row['WLoc'])
            if (counter % 2 == 0):
                diff.append(home) 
                xTrainSeason[counter] = diff
                yTrainSeason[counter] = 1
            else:
                diff.append(-home)
                xTrainSeason[counter] = [ -p for p in diff]
                yTrainSeason[counter] = 0
            counter += 1
        for index, row in tourney.iterrows():
            winning_team = row['WTeamID']
            winning_vector = team_vectors[winning_team]
            losing_team = row['LTeamID']
            losing_vector = team_vectors[losing_team]
            diff = [a - b for a, b in zip(winning_vector, losing_vector)]
            home = 0 #All tournament games are neutral
            if (counter % 2 == 0):
                diff.append(home) 
                xTrainSeason[counter] = diff
                yTrainSeason[counter] = 1
            else:
                diff.append(-home)
                xTrainSeason[counter] = [ -p for p in diff]
                yTrainSeason[counter] = 0
            counter += 1
        xTrain[index_counter:num_games_in_season+index_counter] = xTrainSeason
        yTrain[index_counter:num_games_in_season+index_counter] = yTrainSeason
        index_counter += num_games_in_season
        print ('Finished year:', year)
        if (year in save_years):
            np.save('./Team_Vectors/' + str(year) + 'TeamVectors', team_vectors)
    return xTrain, yTrain

def create_save(years, save_years):
    xTrain, yTrain = train_test(years, save_years)
    np.save('./xTrain_2018.npy', xTrain)
    np.save('./yTrain_2018.npy', yTrain) 

In [22]:
years = range(1993,2018)
# Saves the team vectors for the following years
save_years = range(2014,2018)
if os.path.exists("./xTrain_2018.npy") and os.path.exists("./yTrain_2018.npy"):
    print ('There is already a precomputed xTrain and yTrain.')
    response = input('Do you want to remove these files and create a new training set? (y/n) ')
    if (response == 'y'):
        os.remove("./xTrain_2018.npy")
        os.remove("./yTrain_2018.npy")
        create_save(years, save_years)
    else: 
        print ('Okay, going to exit now.')
else:
    create_save(years, save_years)

There is already a precomputed xTrain and yTrain.
Do you want to remove these files and create a new training set? (y/n) n
Okay, going to exit now.


In [23]:
if os.path.exists("./xTrain_2018.npy") and os.path.exists("./yTrain_2018.npy"):
    xTrain = np.load("./xTrain_2018.npy")
    yTrain = np.load("./yTrain_2018.npy")
    print ("Shape of xTrain:", xTrain.shape)
    print ("Shape of yTrain:", yTrain.shape)
else:
    print ('We need a training set! Run dataPreprocessing.py')
    sys.exit()

Shape of xTrain: (120575, 14)
Shape of yTrain: (120575,)


# Load in the Submission csv


In [24]:
submission = pd.read_csv('./SampleSubmissionStage1.csv')

# All the Models to Try

In [25]:
#model = tree.DecisionTreeClassifier()
#model = tree.DecisionTreeRegressor()
#model = linear_model.LogisticRegression()
#model = linear_model.BayesianRidge()
#model = linear_model.Lasso()
#model = svm.SVC()
#model = svm.SVR()
#model = linear_model.Ridge(alpha = 0.5)
#model = AdaBoostClassifier(n_estimators=100)
#model = GradientBoostingClassifier(n_estimators=100)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)
#model = RandomForestClassifier(n_estimators=64)
#model = KNeighborsClassifier(n_neighbors=39)
#neuralNetwork(10)
#model = VotingClassifier(estimators=[('GBR', model1), ('BR', model2), ('KNN', model3)], voting='soft')
#model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)

# Start Importing Models.

In [26]:
model1 = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)
model = CalibratedClassifierCV(model1)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 16

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

Finished run #0. Accuracy = 0.7615445859872612
Finished run #1. Accuracy = 0.7624734607218684
Finished run #2. Accuracy = 0.7615114118895966
Finished run #3. Accuracy = 0.7616772823779193
Finished run #4. Accuracy = 0.7687101910828026
Finished run #5. Accuracy = 0.7624734607218684
Finished run #6. Accuracy = 0.7616109341825902
Finished run #7. Accuracy = 0.7622412420382165
Finished run #8. Accuracy = 0.7612460191082803
Finished run #9. Accuracy = 0.759952229299363
Finished run #10. Accuracy = 0.7631701167728238
Finished run #11. Accuracy = 0.7646629511677282
Finished run #12. Accuracy = 0.7624734607218684
Finished run #13. Accuracy = 0.7668524416135881
Finished run #14. Accuracy = 0.7622412420382165
Finished run #15. Accuracy = 0.7624402866242038
The average accuracy is:  0.7628300822717622


# Predict a Game.


In [27]:
def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

# Make a Submission

In [28]:
def load_team_vectors(years):
    list_dictionaries = []
    for year in years:
        current_vectors = np.load("./Team_Vectors/" + str(year) + "TeamVectors.npy").item()
        list_dictionaries.append(current_vectors)
    return list_dictionaries

In [29]:
def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)
    print("All Done!")

create_prediction()

Loaded the team vectors
All Done!


# New Model

In [14]:
model = GradientBoostingClassifier(loss='deviance', learning_rate=0.05, max_depth=6, n_estimators=500, max_features='log2')
 
categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 5

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

Finished run #0. Accuracy = 0.7602176220806794
Finished run #1. Accuracy = 0.7635018577494692
Finished run #2. Accuracy = 0.7628715498938429
Finished run #3. Accuracy = 0.7591228768577495
Finished run #4. Accuracy = 0.7563362526539278
The average accuracy is:  0.7604100318471338


In [15]:
def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

In [16]:
def load_team_vectors(years):
    list_dictionaries = []
    for year in years:
        current_vectors = np.load("./Team_Vectors/" + str(year) + "TeamVectors.npy").item()
        list_dictionaries.append(current_vectors)
    return list_dictionaries

In [17]:
def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_GBC.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)

create_prediction()

Loaded the team vectors


# Neural Network Model.


In [18]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

# model = Sequential()

# model = Sequential()
# model.add(Dense(14, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(10, activation='relu'))
# model.add(Dropout(.5))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(4, activation='relu'))
# model.add(Dropout(.15))
# model.add(Dense(1, activation='softmax'))

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 
# categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
#             'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
# accuracy=[]
# num_trials = 1

# for i in range(num_trials):
#     X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
#     results = model.fit(np.array(X_train), np.array(y_train), epochs=20, batch_size=10)
#     #model.fit(np.array(train_X),np.array(train_Y), epochs=20, batch_size=10)

#     preds = model.predict(X_test)

#     preds[preds < .5] = 0
#     preds[preds >= .5] = 1
#     local_accuracy = np.mean(preds == y_test)
#     accuracy.append(local_accuracy)
#     print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
# print("The average accuracy is: ", sum(accuracy)/len(accuracy))

# def predict_game(team_1_vector, team_2_vector, home):
#     diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
#     diff.append(home)
#     # Depending on the model you use, you will either need to return model.predict_proba or model.predict
#     # predict_proba = Linear Reg, Linear SVC
#     # predict = Gradient Boosted

#     return model.predict_proba([diff])[0][1]
#     #return model.predict([diff])[0]

# def load_team_vectors(years):
#     list_dictionaries = []
#     for year in years:
#         current_vectors = np.load("./Team_Vectors/" + str(year) + "TeamVectors.npy").item()
#         list_dictionaries.append(current_vectors)
#     return list_dictionaries

# def create_prediction():
#     if os.path.exists("result.csv"):
#         os.remove("result.csv")
#     # The years that we want to predict for
#     years = range(2014,2018)
#     list_dictionaries = load_team_vectors(years)
#     print ("Loaded the team vectors")
#     results = [[0 for x in range(2)] for x in range(len(submission.index))]
#     for index, row in submission.iterrows():
#         matchup = row['ID']
#         year = int(matchup[0:4])
#         team_vectors = list_dictionaries[year - years[0]]
#         team_1_ID = int(matchup[5:9])
#         team_2_ID = int(matchup[10:14])
#         team_1_vector = team_vectors[team_1_ID]
#         team_2_vector = team_vectors[team_2_ID]
#         pred = predict_game(team_1_vector, team_2_vector, 0)
#         results[index][0] = matchup
#         results[index][1] = pred
#     results = pd.np.array(results)
#     first_row = [[0 for x in range(2)] for x in range(1)]
#     first_row[0][0] = 'ID'
#     first_row[0][1] = 'Pred'
#     with open("result_keras.csv", "w") as f:
#         writer = csv.writer(f)
#         writer.writerows(first_row)
#         writer.writerows(results)

# create_prediction()

Okay so the Neural Net didn't work that well, need to fix that.

# Ada Boost Classifier

In [19]:
model = AdaBoostClassifier(n_estimators=100, learning_rate=0.25, algorithm='SAMME')

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 15

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_ada.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)

create_prediction()

Finished run #0. Accuracy = 0.758359872611465
Finished run #1. Accuracy = 0.752687101910828
Finished run #2. Accuracy = 0.7591228768577495
Finished run #3. Accuracy = 0.7595873142250531
Finished run #4. Accuracy = 0.7573646496815286
Finished run #5. Accuracy = 0.7535164543524416
Finished run #6. Accuracy = 0.7520236199575372
Finished run #7. Accuracy = 0.7519240976645435
Finished run #8. Accuracy = 0.7534501061571125
Finished run #9. Accuracy = 0.7588906581740976
Finished run #10. Accuracy = 0.7511279193205945
Finished run #11. Accuracy = 0.7554405520169851
Finished run #12. Accuracy = 0.7579617834394905
Finished run #13. Accuracy = 0.7530188428874734
Finished run #14. Accuracy = 0.758426220806794
The average accuracy is:  0.7555268046709129
Loaded the team vectors


# Random Forest Classifier

In [20]:
model = RandomForestClassifier(n_estimators=125, criterion='entropy', max_features='log2', min_samples_split=2)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 16

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_rf.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)

create_prediction()

Finished run #0. Accuracy = 0.7283041401273885
Finished run #1. Accuracy = 0.7247545116772823
Finished run #2. Accuracy = 0.7231621549893843
Finished run #3. Accuracy = 0.726015127388535
Finished run #4. Accuracy = 0.7283704883227177
Finished run #5. Accuracy = 0.7266786093418259
Finished run #6. Accuracy = 0.725583864118896
Finished run #7. Accuracy = 0.7274084394904459
Finished run #8. Accuracy = 0.7258492569002123
Finished run #9. Accuracy = 0.7278728768577495
Finished run #10. Accuracy = 0.7287354033970276
Finished run #11. Accuracy = 0.7240246815286624
Finished run #12. Accuracy = 0.7274747876857749
Finished run #13. Accuracy = 0.7228304140127388
Finished run #14. Accuracy = 0.727109872611465
Finished run #15. Accuracy = 0.7245886411889597
The average accuracy is:  0.7261727043524416
Loaded the team vectors


# Logistic Regression,  L1

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
model = LogisticRegression(penalty='l1', C=0.5, max_iter=200)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 16

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_rf.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)

create_prediction()

Finished run #0. Accuracy = 0.7627720276008493
Finished run #1. Accuracy = 0.7627720276008493
Finished run #2. Accuracy = 0.7611133227176221
Finished run #3. Accuracy = 0.7650942144373672
Finished run #4. Accuracy = 0.7634355095541401
Finished run #5. Accuracy = 0.7628052016985138
Finished run #6. Accuracy = 0.762506634819533
Finished run #7. Accuracy = 0.7625398089171974
Finished run #8. Accuracy = 0.7655586518046709
Finished run #9. Accuracy = 0.7605161889596603
Finished run #10. Accuracy = 0.7621085456475584
Finished run #11. Accuracy = 0.7597200106157113
Finished run #12. Accuracy = 0.7604498407643312
Finished run #13. Accuracy = 0.7606488853503185
Finished run #14. Accuracy = 0.7617104564755839
Finished run #15. Accuracy = 0.7657245222929936
The average accuracy is:  0.7624672405785562
Loaded the team vectors


# Logistic Regression, L2

In [23]:
model = LogisticRegression(penalty='l2', C=0.5, tol=0.00001, max_iter=500)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 16

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_l2.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)
    print('All Done!')
create_prediction()

Finished run #0. Accuracy = 0.7654259554140127
Finished run #1. Accuracy = 0.7587579617834395
Finished run #2. Accuracy = 0.7624402866242038
Finished run #3. Accuracy = 0.7596536624203821
Finished run #4. Accuracy = 0.7607152335456475
Finished run #5. Accuracy = 0.7609474522292994
Finished run #6. Accuracy = 0.7621417197452229
Finished run #7. Accuracy = 0.7589570063694268
Finished run #8. Accuracy = 0.7576632165605095
Finished run #9. Accuracy = 0.7602507961783439
Finished run #10. Accuracy = 0.7627056794055201
Finished run #11. Accuracy = 0.7630705944798302
Finished run #12. Accuracy = 0.7571324309978769
Finished run #13. Accuracy = 0.7620090233545648
Finished run #14. Accuracy = 0.7621417197452229
Finished run #15. Accuracy = 0.7608811040339702
The average accuracy is:  0.7609308651804672
Loaded the team vectors
All Done!


# KNearestNeighbors

In [24]:
model = KNeighborsClassifier(n_neighbors=6)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 16

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_knn.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)
    print('All Done!')
create_prediction()

Finished run #0. Accuracy = 0.7226977176220807
Finished run #1. Accuracy = 0.7253184713375797
Finished run #2. Accuracy = 0.7222996284501062
Finished run #3. Accuracy = 0.7237592887473461
Finished run #4. Accuracy = 0.7190817409766455
Finished run #5. Accuracy = 0.7249203821656051
Finished run #6. Accuracy = 0.7231289808917197
Finished run #7. Accuracy = 0.7236597664543525
Finished run #8. Accuracy = 0.7223328025477707
Finished run #9. Accuracy = 0.7253848195329087
Finished run #10. Accuracy = 0.7268444798301487
Finished run #11. Accuracy = 0.7207736199575372
Finished run #12. Accuracy = 0.7196788747346072
Finished run #13. Accuracy = 0.7274747876857749
Finished run #14. Accuracy = 0.7259156050955414
Finished run #15. Accuracy = 0.7246218152866242
The average accuracy is:  0.7236182988322718
Loaded the team vectors
All Done!


# Gradient Boosting Regressor

In [25]:
model = GradientBoostingRegressor(loss='lad', learning_rate=.001, n_estimators=500, max_depth=6, min_samples_split=4)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 16

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    #return model.predict_proba([diff])[0][1]
    return model.predict([diff])[0]

def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_knn.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)
    print('All Done!')
create_prediction()

Finished run #0. Accuracy = 0.49595276008492567
Finished run #1. Accuracy = 0.4973792462845011
Finished run #2. Accuracy = 0.498473991507431
Finished run #3. Accuracy = 0.4971470276008493
Finished run #4. Accuracy = 0.4984408174097665
Finished run #5. Accuracy = 0.49727972399150744
Finished run #6. Accuracy = 0.49910429936305734
Finished run #7. Accuracy = 0.4993033439490446
Finished run #8. Accuracy = 0.49850716560509556
Finished run #9. Accuracy = 0.49980095541401276
Finished run #10. Accuracy = 0.49920382165605093
Finished run #11. Accuracy = 0.4958200636942675
Finished run #12. Accuracy = 0.4963508492569002
Finished run #13. Accuracy = 0.4955878450106157
Finished run #14. Accuracy = 0.49781050955414013
Finished run #15. Accuracy = 0.49913747346072185
The average accuracy is:  0.4978312433651805
Loaded the team vectors
All Done!


# Gradient Boosting Classifier

In [26]:
model = GradientBoostingClassifier()

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 16

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_gbclass.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)
    print('All Done!')
create_prediction()

Finished run #0. Accuracy = 0.7660230891719745
Finished run #1. Accuracy = 0.7651273885350318
Finished run #2. Accuracy = 0.7606820594479831
Finished run #3. Accuracy = 0.7641653397027601
Finished run #4. Accuracy = 0.7612791932059448
Finished run #5. Accuracy = 0.76453025477707
Finished run #6. Accuracy = 0.7628383757961783
Finished run #7. Accuracy = 0.7583266985138004
Finished run #8. Accuracy = 0.7612460191082803
Finished run #9. Accuracy = 0.7649615180467091
Finished run #10. Accuracy = 0.7651273885350318
Finished run #11. Accuracy = 0.7619426751592356
Finished run #12. Accuracy = 0.7594214437367304
Finished run #13. Accuracy = 0.7618763269639066
Finished run #14. Accuracy = 0.7639662951167728
Finished run #15. Accuracy = 0.7583930467091295
The average accuracy is:  0.7624941945329088
Loaded the team vectors
All Done!


# NuSVC


In [ ]:
from sklearn.svm import NuSVC

In [ ]:
model = NuSVC()

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Seed','SOS',\
            'SRS', 'RPG', 'SPG', 'Tourney Appearances','Location']
accuracy=[]
num_trials = 16

for i in range(num_trials):
    X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    local_accuracy = np.mean(preds == y_test)
    accuracy.append(local_accuracy)
    print("Finished run #" + str(i) + ". Accuracy = " + str(local_accuracy))
print("The average accuracy is: ", sum(accuracy)/len(accuracy))

def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    # Depending on the model you use, you will either need to return model.predict_proba or model.predict
    # predict_proba = Linear Reg, Linear SVC
    # predict = Gradient Boosted

    return model.predict_proba([diff])[0][1]
    #return model.predict([diff])[0]

def create_prediction():
    if os.path.exists("result.csv"):
        os.remove("result.csv")
    # The years that we want to predict for
    years = range(2014,2018)
    list_dictionaries = load_team_vectors(years)
    print ("Loaded the team vectors")
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup = row['ID']
        year = int(matchup[0:4])
        team_vectors = list_dictionaries[year - years[0]]
        team_1_ID = int(matchup[5:9])
        team_2_ID = int(matchup[10:14])
        team_1_vector = team_vectors[team_1_ID]
        team_2_vector = team_vectors[team_2_ID]
        pred = predict_game(team_1_vector, team_2_vector, 0)
        results[index][0] = matchup
        results[index][1] = pred
    results = pd.np.array(results)
    first_row = [[0 for x in range(2)] for x in range(1)]
    first_row[0][0] = 'ID'
    first_row[0][1] = 'Pred'
    with open("result_nusvc.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(first_row)
        writer.writerows(results)
    print('All Done!')
create_prediction()

Finished run #0. Accuracy = 0.7038880042462845
